In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from bs4 import NavigableString
import requests
import pandas as pd
import re
import collections
import time
from sklearn.preprocessing import MultiLabelBinarizer
import os
import sys
import zipfile
import glob

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

In [2]:
source_dir = r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Files'


def get_text_from_clean(text, liste, direction= "right"):

    pairs = []
   
    if direction == "right":
        for item in liste:
            try:
                if item in text:
                    pairs.append([item, text.split(item)[1].split()[0]])
                else:
                    pairs.append([item, "none"])
            except:
                pairs.append([item, "none"])
    if direction == "left":
        for item in liste:
            try:
                if item in text:
                    pairs.append([item, text.split(item)[0].split()[-1]])
                else:
                     pairs.append([item, "none"])   
            except:
                pairs.append([item, "none"])
    
    return pairs

def clean_string(liste):
    liste = Flatten(liste)
    liste = " ".join(liste)
    liste = " ".join(liste.split())
    return liste

def get_text_from_html(bs, class_name):
    string_list =[]
    soup = bs.find_all(class_=class_name)
    for entry in soup:
        string_list.append(entry.text)
    return string_list

def Flatten(ul):
    fl = []
    for i in ul:
        if type(i) is list:
            fl += Flatten(i)
        else:
            fl += [i]
    return fl

def get_all_data_from_site(bs, link):
      
    names = ["Wohnung","Zimmergröße","Sonstige","Nebenkosten","Miete","Gesamtmiete","Kaution","Ablösevereinbarung"]
    my_list = get_text_from_html(bs,"col-sm-12 hidden-xs")
    my_list = clean_string(my_list)
    dict1 = dict(get_text_from_clean(my_list,names,"left"))
       
    names = ["frei ab: ", "frei bis: "]
    my_list = get_text_from_html(bs,"col-sm-3")
    my_list = clean_string(my_list)
    dict2 = dict(get_text_from_clean(my_list,names,"right"))
    
    names = [" Zimmer in "]
    my_list = get_text_from_html(bs,"col-sm-6")
    my_list = clean_string(my_list)
    dict3 = dict(get_text_from_clean(my_list,names,"right"))
       
    names = ["Frau ","Frauen ", "Mann)", " Männer)", " Männer und"]
    my_list = get_text_from_html(bs,"col-sm-6")
    my_list = clean_string(my_list)
    dict4 = dict(get_text_from_clean(my_list,names,"left"))
    
    my_list = get_text_from_html(bs,"ul-detailed-view-datasheet print_text_left")
    my_list = [x.strip() for x in my_list]
    try:
        dict5 = dict(get_text_from_clean(my_list[1],["zwischen"],"left"))
    except:
        dict5 = dict(get_text_from_clean(my_list,["zwischen"],"left"))
        
                                     
    my_list = get_text_from_html(bs,"ul-detailed-view-datasheet print_text_left")
    my_list = [x.strip() for x in my_list]
    try:
        dict8 = dict(get_text_from_clean(my_list[1],["Geschlecht"],"right"))
    except:
        dict8 = dict(get_text_from_clean(my_list,["Geschlecht"],"right"))

    
    item_list = ["glyphicons glyphicons-bath-bathtub noprint",
    "glyphicons glyphicons-wifi-alt noprint",
    "glyphicons glyphicons-car noprint",
    "glyphicons glyphicons-fabric noprint",
    "glyphicons glyphicons-display noprint",
    "glyphicons glyphicons-folder-closed noprint",
    "glyphicons glyphicons-mixed-buildings noprint",
    "glyphicons glyphicons-building noprint",
    "glyphicons glyphicons-bus noprint",
    "glyphicons glyphicons-electricity noprint",
    "glyphicons glyphicons-fire noprint"]
    data_list = []
    for item in item_list:
        try:
            data_list.append([item[22:-8], clean_string([bs.find(class_=item).next_sibling.next_sibling.next_sibling])])
        except:
            data_list.append([item[22:-8],"none"])
    dict6 = dict(data_list)
    
    liste= get_text_from_html(bs,"col-sm-4 mb10")
    adress_string = clean_string(liste).replace("Adresse ","").replace("Umzugsfirma beauftragen1","").replace("Umzugsfirma beauftragen 1","")
    dict7 = {"Adresse":adress_string, "Link": link}
    
    dict_list =[dict1,dict2,dict3,dict4,dict5,dict8, dict6,dict7,dict7]
    for item in dict_list:
        dict1.update(item)
    return dict1

def get_bs_from_html(html):
    return BeautifulSoup(html.text, "html.parser")

def get_bs_from_http(link):
    html = requests.get(link)
    return BeautifulSoup(html.text, "html.parser")

def get_html_request(link):
    return requests.get(link)

def get_html_from_scraper(link):
    payload = {'api_key': '5002b3eb47039688a731795808c619fd', 'url':link}
    html = requests.get('http://api.scraperapi.com', params=payload)
    return html

def get_bs_from_http_scraper(link):
    payload = {'api_key': '318ea492c3446fdf31906e1c267cd865', 'url':link}
    html = requests.get('http://api.scraperapi.com', params=payload)
    return BeautifulSoup(html.text, "html.parser")


def get_all_links_from_site(bs):
    try:
        all_links = []
        classes = ["listenansicht1 listenansicht-inactive offer_list_item","listenansicht0 listenansicht-inactive offer_list_item",
                  "listenansicht0 offer_list_item", "listenansicht1 offer_list_item"]
        
        for scrape_class in classes:
            links = bs.findAll(class_=scrape_class)
            for link in links:
                all_links.append(link["adid"][31:])
    except:
        print("something went wrong with get_all_links")
    return all_links

def get_all_links(nr_min_sites = 0, nr_max_sites = 0, sleep_time =0, scraper = get_bs_from_http_scraper):
    linklist = []
    for i in range(nr_min_sites,nr_max_sites):
        try:
            url = 'https://www.wg-gesucht.de/wg-zimmer-in-Frankfurt-am-Main.41.0.0.'+ str(i) +'.html'
            linklist.extend(get_all_links_from_site(scraper(url)))
            print(f"{i+1-nr_min_sites} from {nr_max_sites-nr_min_sites} Pages loaded. Thats {(i-nr_min_sites+1)/(nr_max_sites-nr_min_sites):.2%}.\
            Linklist now has {len(linklist)} rows (expexted {(i+1-nr_min_sites)*20})", end='\r')        
            time.sleep(sleep_time)
        except:
            pass
    return linklist   

def merge_dicts(dic1,dic2):
    try:
        dic3 = dict(dic2)
        for k, v in dic1.items():
            dic3[k] = Flatten([dic3[k], v]) if k in dic3 else v
        return dic3
    except:
        return dic1

def csv_files(source_dir):
    for filename in os.listdir(source_dir):
        if filename.endswith('.txt'):
            yield filename

def zip_files(source_dir):
    for filename in os.listdir(source_dir):
        if filename.endswith('.zip'):
            yield filename

def get_bot_and_outdated_links():
    deleted_txt = "Vermutlich ist das Objekt bereits vergeben."
    cap_txt ="dass kein Bot die Website automatisiert aufruft."

    source_dir = r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Files'
    theFiles = list(os.listdir(source_dir))

    file_list_del = []
    file_list_bot =[]
    theDict = dict()
    for i in theFiles: #Calculate size for all files here. 
        theStats = os.stat(source_dir + "\\"+i)
        theDict[i] = theStats
        if theDict[i].st_size < 85000 and theDict[i].st_size > 60000:
            f = source_dir + "\\"+ i
            with open(f, "r") as file:
                a = file.read()
                if (deleted_txt in a):
                    file_list_del.append(i[:-4])
                if (cap_txt in a):
                    file_list_bot.append(i[:-4])

    for item in file_list_del:
        f = source_dir + "\\"+ item + ".txt"
        os.remove(f)

    for item in file_list_bot:
        f = source_dir + "\\"+ item + ".txt"
        os.remove(f)    

    t1 = pd.read_csv(r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Housing\Outdated_links.csv')
    t2 = pd.DataFrame(file_list_del, columns =['Links'])
    t1 = t1.append(t2).drop_duplicates(keep="first")
    t1.to_csv(r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Housing\Outdated_links.csv', index=False)

    t1 = pd.read_csv(r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Housing\Bot_links.csv')
    t2 = pd.DataFrame(file_list_bot, columns =['Links'])
    t1 = t1.append(t2).drop_duplicates(keep="first")
    t1.to_csv(r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Housing\Bot_links.csv', index=False)

def zip_my_files():
    source_dir = r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Files'
    dest_dir = r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Files'
    os.chdir(dest_dir)  # To work around zipfile limitations
    for csv_filename in csv_files(source_dir):
        file_root = os.path.splitext(csv_filename)[0]
        zip_file_name = file_root + '.zip'
        zip_file_path = os.path.join(dest_dir, zip_file_name)
        with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zf:
            zf.write(csv_filename)
        os.remove(r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Files\\' + csv_filename)
        
def get_housing_links():
    
    get_bot_and_outdated_links()
    zipped_files = []
    for csv_filename in zip_files(source_dir):
        zipped_files.append(csv_filename[:-4])
    zipped_files = pd.DataFrame(zipped_files, columns =['Links'])
    zipped_files["Links"] = zipped_files["Links"].str.lower()

    house_links = pd.read_csv(r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Housing\All_Housing_Links.csv')
    house_links["Links"] = house_links["Links"].str.lower()
    outdated_links = pd.read_csv(r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Housing\Outdated_links.csv')
    outdated_links["Links"] = outdated_links["Links"].str.lower()
    
    house_links["Links"] = house_links["Links"].str.slice(0,-5)
    house_links = house_links["Links"].append(zipped_files["Links"])
    house_links = house_links.append(outdated_links["Links"])
    
    house_links = house_links.drop_duplicates(keep=False)
    return house_links            

In [4]:
#initial_links = pd.DataFrame(new_links, columns=["Links"])
#initial_links.to_csv (r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Housing\All_Housing_Links.csv', index = None, header=True)

In [9]:
get_all_links(nr_min_sites = 0,nr_max_sites= 1, sleep_time = 0, scraper= get_bs_from_http_scraper)

['Westend-Nord.7697498.html',
 'Innenstadt.7692275.html',
 'Nordend-West.7418667.html',
 'Frankfurt-am-Main.4331973.html',
 'Frankfurt-am-Main.5892499.html',
 'Nordend-West.1829682.html',
 'Westend-Nord.7695069.html',
 'Dornbusch.6832992.html',
 'Westend-Nord.7693038.html',
 'Nordend-West.1098997.html',
 'Westend-Nord.7695351.html',
 'Westend-Nord.7952174.html',
 'Westend-Nord.7743282.html',
 'Frankfurt.4644370.html',
 'Westend-Sud.7692756.html',
 'Westend-Nord.7700915.html',
 'Westend-Nord.8048404.html',
 'Dornbusch.6354740.html',
 'Ostend.6206905.html',
 'Gallus.4744790.html']

In [10]:
##### Load old links
old_links = pd.read_csv(r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Housing\All_Housing_Links.csv', usecols=["Links"])
##### Scrape new links
new_links = get_all_links(nr_min_sites = 0,nr_max_sites= 30, sleep_time = 1, scraper= get_bs_from_http)
new_links = pd.DataFrame(new_links, columns=["Links"])
#### Combine and save
combined = pd.concat([old_links, new_links]).drop_duplicates(keep="first")
combined.to_csv (r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Housing\All_Housing_Links.csv', index = True, header=True)


In [11]:
print(len(old_links))
print(len(new_links))
print(len(combined))


14772
600
15004


In [ ]:
house_links = get_housing_links()
zip_my_files()
cap_txt ="Alle Webseiten nutzen daher Captchas, um zu prüfen, ob beispielsweise ein Formular"
cap_txt2 = "Vermutlich ist das Objekt bereits vergeben."
cap_txt3 ="Wenn Sie glauben, dass dies ein Fehler im System ist, dann schreiben Sie doch bitte eine kurze Nachricht"

for i in range(0,4498):
    if len(house_links) < 1:
        break
    name = house_links.iloc[i]
    stem = "https://www.wg-gesucht.de/wg-zimmer-in-Frankfurt-am-Main-"
    link =  stem + name + ".html"
    path = r"C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Files\\"+name + ".txt"
    time.sleep(120)
    try:
        r = get_html_request(link)
        if cap_txt in r.text:
            print("bot",  end='\r')
        elif cap_txt2 in r.text:
            print("vergeben",  end='\r')
        elif cap_txt3 in r.text:
            print("nicht gefunden",  end='\r')
        else:
            print(link,  end='\r')
        with open(path, 'w', encoding="utf-8") as file:
            file.write(r.text) 
    except:
        print("error",  end='\r')
    
      


In [3]:
link = "https://www.wg-gesucht.de/wg-zimmer-in-Frankfurt-am-Main-Innenstadt.7692275.html"
bs = get_bs_from_http(link)

In [4]:
a = get_all_data_from_site(bs, link)

In [18]:
cols = ['wohnung', 'zimmergröße', 'sonstige', 'nebenkosten', 'miete', 'gesamtmiete', 'kaution',
     'ablösevereinbarung', 'frei_ab', 'frei_bis', 'personen', 'männlich', 'weiblich', 'insgesamt', 'geschlecht', 'geschlecht2', 'bath-bathtub', 'wifi-alt', 'car',
     'fabric', 'display', 'folder-closed', 'buildings', 'stock', 'bus', 'bed',
     'fire', 'adresse', 'link', 'taeglich', 'baujahr','rauchen','zweck_wg','keine_zweck_wg','beruf_wg','gemischt_wg','studenten_wg','frauen_wg', 'azubi_wg', 'zwischenmiete', 'renoviert', 'gender',
           'neuwertig', 'neu_eingerichtet', 'praktikant', 'frisch_gestrichen',
           'studentenverbindung', 'wohnheim']

In [22]:
a.items()

dict_items([('Wohnung', '90m²'), ('Zimmergröße', '25m²'), ('Sonstige', 'n.a.'), ('Nebenkosten', '100€'), ('Miete', '700€'), ('Gesamtmiete', '800€'), ('Kaution', 'n.a.'), ('Ablösevereinbarung', 'n.a.'), ('frei ab: ', '01.09.2020'), ('frei bis: ', 'none'), (' Zimmer in ', '3er'), ('Frau ', '1'), ('Frauen ', 'none'), ('Mann)', 'none'), (' Männer)', 'none'), (' Männer und', 'none'), ('zwischen', 'none'), ('Geschlecht', 'egal'), ('bath-bathtub', 'Dusche'), ('wifi-alt', 'Flatrate, WLAN schneller als 100 Mbit/s'), ('car', 'gute Parkmöglichkeiten'), ('fabric', 'Parkett'), ('display', 'Kabel'), ('folder-closed', 'Waschmaschine, Spülmaschine, Terrasse, Balkon, Keller, Fahrradkeller, Aufzug'), ('mixed-buildings', 'none'), ('building', 'none'), ('bus', 'none'), ('electricity', 'none'), ('fire', 'none'), ('Adresse', 'Weserstraße 7 60329 Frankfurt am Main Innenstadt '), ('Link', 'https://www.wg-gesucht.de/wg-zimmer-in-Frankfurt-am-Main-Innenstadt.7692275.html')])

In [28]:
b = pd.DataFrame([a])

In [7]:
val = [13.013,12.65,8.76,6.719,6.304,5.811,4.362,4.123,3.352,3.066,2.785,2.508,2.479,2.452,2.356,2.083,1.858,1.491,1.197,1.159,1.1,1.041,0.99,0.838,0.8,0.762,0.654,0.619,0.571,0.517,0.515,0.504,0.454,0.414,0.402,0.38,0.308,0.221,0.132,0.074,0.066,0.054,0.054,0.003]


In [8]:
val2 = []
for i in val:
    val2.append("test")

In [9]:
val2

['test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test']

2.2